In [ ]:
# Criterion: R2 Score
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
import time
import os

class StableSFS:
    def __init__(self, n_features_to_select=7, n_estimators=None, max_depth=None, cv=10, random_state=1, verbose=True):
        self.n_features_to_select = n_features_to_select
        self.max_depth = max_depth
        self.cv = cv
        self.random_state = random_state
        self.initial_n_estimators = n_estimators
        self.n_estimators = n_estimators
        self.selected_features_ = None
        self.feature_scores_ = {}
        self.performance_log_ = []
        self.verbose = verbose

    def _find_optimal_estimators(self, X, y_input, max_estimators=1000, step=50):
        y = y_input
        if isinstance(y_input, pd.Series):
            y = y_input.values
        np.random.seed(self.random_state)

        if self.verbose:
            print(f"\n[Seed {self.random_state}] Finding optimal n_estimators...")

        estimator_range = range(50, max_estimators + 1, step)
        best_score = -np.inf
        best_n = 100
        kfold = KFold(n_splits=self.cv, shuffle=True, random_state=self.random_state)

        for n in estimator_range:
            fold_scores = []
            for train_idx, val_idx in kfold.split(X):
                X_tr, y_tr = X.iloc[train_idx], y[train_idx]
                X_val, y_val = X.iloc[val_idx], y[val_idx]

                model = RandomForestRegressor(
                    n_estimators=n,
                    max_depth=self.max_depth,
                    random_state=self.random_state,
                    n_jobs=1
                )
                model.fit(X_tr, y_tr)
                pred = model.predict(X_val)
                fold_scores.append(r2_score(y_val, pred))

            mean_score = np.mean(fold_scores)

            if self.verbose:
                print(f"  n_estimators={n:3d} | R²={mean_score:.4f}")

            if mean_score > best_score:
                best_score = mean_score
                best_n = n
            elif mean_score == best_score and n < best_n:
                if self.verbose:
                    print(f"  n_estimators={n:3d} | R² is the same ({mean_score:.4f}), preferring smaller n_estimators (was {best_n})")
                best_n = n

        if self.verbose:
            print(f"[Seed {self.random_state}] Optimal n_estimators: {best_n} (R²={best_score:.4f})")

        return best_n

    def _evaluate_feature_stability(self, X, y_input, feature_set):
        """Evaluate the stability and performance of a given feature subset."""
        y = y_input
        if isinstance(y_input, pd.Series):
            y = y_input.values

        np.random.seed(self.random_state)
        kf = KFold(n_splits=self.cv, shuffle=True, random_state=self.random_state)
        fold_scores = []
        fold_importances = {feature: [] for feature in feature_set}

        for train_idx, val_idx in kf.split(X):
            X_train_fold, y_train_fold = X.iloc[train_idx][feature_set], y[train_idx]
            X_val_fold, y_val_fold = X.iloc[val_idx][feature_set], y[val_idx]

            model = RandomForestRegressor(
                n_estimators=self.n_estimators,
                max_depth=self.max_depth,
                random_state=self.random_state,
                n_jobs=1
            )
            model.fit(X_train_fold, y_train_fold)
            y_pred = model.predict(X_val_fold)
            fold_scores.append(r2_score(y_val_fold, y_pred))
            
            importances = dict(zip(feature_set, model.feature_importances_))
            for feature in feature_set:
                fold_importances[feature].append(importances[feature])

        mean_score = np.mean(fold_scores)
        std_score = np.std(fold_scores)
        
        feature_stability = {}
        for feature in feature_set:
            mean_importance = np.mean(fold_importances[feature])
            std_importance = np.std(fold_importances[feature])
            stability_score = mean_importance / (std_importance + 1e-10) # Not used, but calculated
            feature_stability[feature] = {
                'mean_importance': mean_importance,
                'std_importance': std_importance,
                'stability_score': stability_score
            }
        return mean_score, std_score, feature_stability

    def fit(self, X, y_input):
        """Execute stable feature selection - forcing selection of a specified number of features."""
        y = y_input
        if isinstance(y_input, pd.Series):
            y = y_input.values

        np.random.seed(self.random_state)

        if self.verbose:
            print(f"\n===== [Seed {self.random_state}] Starting Feature Selection =====")

        if self.initial_n_estimators is None:
            self.n_estimators = self._find_optimal_estimators(X, y)
        elif self.verbose:
            print(f"[Seed {self.random_state}] Using specified n_estimators={self.n_estimators}")

        selected_features = []
        remaining_features = sorted(X.columns.tolist())

        if "pm" in remaining_features:
            selected_features.append("pm")
            remaining_features.remove("pm")
            if self.verbose:
                print(f"[Seed {self.random_state}] Forcing first feature: pm")
        
        # SFS main loop
        for i in range(len(selected_features), self.n_features_to_select):
            if not remaining_features or len(selected_features) >= self.n_features_to_select:
                if self.verbose and not remaining_features and len(selected_features) < self.n_features_to_select:
                    print(f"[Seed {self.random_state}] No more features to select. {len(selected_features)} features selected.")
                break

            if self.verbose:
                print(f"\n[Seed {self.random_state}] Feature Selection Round {i+1} (Selected: {len(selected_features)})")
                print(f"  Current feature set: {selected_features}")

            best_score_for_round = -np.inf
            best_feature_for_round = None
            current_evaluations = {}

            for feature_to_add in sorted(remaining_features):
                candidate_set = selected_features + [feature_to_add]
                score, score_std, stability_info = self._evaluate_feature_stability(
                    X, y, candidate_set
                )
                current_evaluations[feature_to_add] = {
                    'score': score,
                    'std': score_std,
                    'stability': stability_info[feature_to_add]['stability_score'] if feature_to_add in stability_info else np.nan
                }
                if self.verbose:
                    print(f"  Evaluating feature: {feature_to_add:15s} | R²: {score:.4f} ± {score_std:.4f}")

                if score > best_score_for_round:
                    best_score_for_round = score
                    best_feature_for_round = feature_to_add
                elif score == best_score_for_round and best_feature_for_round is not None and feature_to_add < best_feature_for_round:
                    best_feature_for_round = feature_to_add

            if best_feature_for_round is None:
                if self.verbose:
                    print(f"[Seed {self.random_state}] No suitable feature found among remaining or limit reached.")
                break

            selected_features.append(best_feature_for_round)
            remaining_features.remove(best_feature_for_round)

            if self.verbose:
                print(f"  [Seed {self.random_state}] Round {i+1} selected feature: {best_feature_for_round} (R²: {best_score_for_round:.4f})")

            self.performance_log_.append({
                'iteration': i + 1,
                'selected': best_feature_for_round,
                'current_set': selected_features.copy(),
                'score': best_score_for_round,
                'evaluations': current_evaluations
            })

            for feat_eval, eval_data in current_evaluations.items():
                if feat_eval not in self.feature_scores_:
                    self.feature_scores_[feat_eval] = []
                self.feature_scores_[feat_eval].append(eval_data)

        self.selected_features_ = selected_features

        if self.verbose:
            print(f"\n[Seed {self.random_state}] Feature selection complete!")
            print(f"  Final selected features: {self.selected_features_}")
            print("-" * 80)

        return self

def multi_seed_stable_selection(X, y_input, n_seeds=400, target_features=7, start_seed=1, verbose=False):
    y = y_input
    if isinstance(y_input, pd.Series):
        y = y_input.values

    seed_results = []
    all_selected_features = []
    feature_counter = Counter()
    collected_n_estimators = []

    timestamp = time.strftime('%Y%m%d_%H%M%S')
    # Change the output directory name here
    output_dir_base = "sfspcc400times"
    log_dir = f"{output_dir_base}_{timestamp}" 
    os.makedirs(log_dir, exist_ok=True)
    log_file_path = os.path.join(log_dir, "sfs_run.log")

    with open(log_file_path, "w", encoding="utf-8") as f:
        f.write(f"=== Multi-Seed Stable Feature Selection - Start Time: {time.strftime('%Y-%m-%d %H:%M:%S')} ===\n")
        f.write(f"Number of seeds: {n_seeds}, Target features: {target_features}, Start seed: {start_seed}\n\n")

    if verbose:
        print(f"Executing stable feature selection with {n_seeds} different random seeds...")
        print(f"Logs will be saved to: {log_file_path}")

    progress_bar = tqdm(range(n_seeds), desc="SFS Progress")

    for i in progress_bar:
        current_seed = start_seed + i
        progress_bar.set_description(f"Processing seed {current_seed}")

        selector = StableSFS(
            n_features_to_select=target_features,
            cv=10,
            random_state=current_seed,
            verbose=verbose
        )
        selector.fit(X, y)

        current_run_selected_features = selector.selected_features_ if selector.selected_features_ is not None else []
        all_selected_features.append(current_run_selected_features)

        if selector.n_estimators is not None:
            collected_n_estimators.append(selector.n_estimators)

        for feature_name in current_run_selected_features:
            feature_counter[feature_name] += 1

        seed_results.append({
            'seed': current_seed,
            'features': current_run_selected_features,
            'n_estimators': selector.n_estimators
        })

        with open(log_file_path, "a", encoding="utf-8") as f:
            f.write(f"\nSeed {current_seed} Results:\n")
            f.write(f"  n_estimators: {selector.n_estimators}\n")
            f.write(f"  Selected Features: {', '.join(current_run_selected_features)}\n")
            for j, log_entry in enumerate(selector.performance_log_):
                f.write(f"  Round {j+1}: Selected '{log_entry['selected']}' (CV R²: {log_entry['score']:.4f})\n")

    if verbose:
        print("\n===== Multi-Seed Stable Feature Selection Results =====")
        print(f"Total runs: {n_seeds}")

    sorted_features = sorted(feature_counter.items(), key=lambda x: (-x[1], x[0]))

    if verbose:
        print("\nFeature Selection Frequency (descending):")
        for feature_name, count in sorted_features:
            percentage = (count / n_seeds) * 100
            print(f"{feature_name}: {count}/{n_seeds} ({percentage:.1f}%)")

    most_stable_features = [f[0] for f in sorted_features[:target_features]]

    if verbose:
        print(f"\nRecommended most stable {target_features}-feature set:")
        print(most_stable_features)

    with open(log_file_path, "a", encoding="utf-8") as f:
        f.write("\n===== Final Summary =====\n")
        f.write("Feature Selection Frequency (descending):\n")
        for feature_item_log, count_log in sorted_features:
            percentage = (count_log / n_seeds) * 100
            f.write(f"{feature_item_log}: {count_log}/{n_seeds} ({percentage:.1f}%)\n")
        f.write(f"\nRecommended most stable {target_features}-feature set:\n")
        f.write((", ".join(most_stable_features) if most_stable_features else "None") + "\n")
        f.write(f"\nRun End Time: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")

    if sorted_features:
        plt.figure(figsize=(12, 6))
        plot_feature_names_list = [f[0] for f in sorted_features]
        plot_counts_list = [f[1] for f in sorted_features]
        bars = plt.bar(plot_feature_names_list, plot_counts_list)
        plt.axhline(y=n_seeds/2, color='r', linestyle='--', label='50% Selection Rate')
        for i, feature_name_in_plot in enumerate(plot_feature_names_list):
            if feature_name_in_plot in most_stable_features:
                bars[i].set_color('green')
        plt.xlabel('Features')
        plt.ylabel('Selection Frequency')
        plt.title('Feature Selection Frequency Distribution')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.legend(['50% Selection Rate', 'Most Stable Features'])
        plt.savefig(os.path.join(log_dir, 'feature_selection_frequency.png'))
        plt.close()
        if verbose: print(f"Plot saved to: {os.path.join(log_dir, 'feature_selection_frequency.png')}")

    if sorted_features:
        result_df = pd.DataFrame({
            'feature': [f[0] for f in sorted_features],
            'selection_frequency': [f[1] for f in sorted_features],
            'percentage': [f[1]/n_seeds*100 for f in sorted_features]
        })
        result_df.to_csv(os.path.join(log_dir, 'feature_selection_results.csv'), index=False)
        if verbose: print(f"Results CSV saved to: {os.path.join(log_dir, 'feature_selection_results.csv')}")

    if seed_results:
        seed_df = pd.DataFrame(seed_results)
        seed_df['features'] = seed_df['features'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
        seed_df.to_csv(os.path.join(log_dir, 'seed_details.csv'), index=False)
        if verbose: print(f"Seed details CSV saved to: {os.path.join(log_dir, 'seed_details.csv')}")

    return most_stable_features, sorted_features, seed_results

if __name__ == "__main__":
    print("===== Multi-Seed SFS Feature Selection based on Random Forest =====")

    try:
        data_path = "/Users/yangmingyue/Desktop/78/降维后PCC/ABO478PCC.csv" # Data file path
        data = pd.read_csv(data_path)
        print(f"Data dimensions: {data.shape[0]} rows x {data.shape[1]} columns")

        target_column = "TCF" # Target variable column name
        if target_column not in data.columns:
            raise ValueError(f"Column not found: {target_column}")

        X = data.drop(columns=[target_column])
        y_series = data[target_column]

    except FileNotFoundError:
        print(f"Error: Data file '{data_path}' not found.")
        exit(1)
    except ValueError as ve:
        print(f"Data processing error: {ve}")
        exit(1)
    except Exception as e:
        print(f"An error occurred while loading data: {e}")
        exit(1)

    if X.empty or y_series.empty:
        print("Error: Feature set X or target set y is empty.")
        exit(1)
    if X.shape[0] < 5:
        print(f"Error: Not enough data samples ({X.shape[0]} rows). At least 5 are needed for effective cross-validation.")
        exit(1)

    print(f"Features: {X.shape[1]} columns, Samples: {X.shape[0]} rows")

    # Change the number of seeds here
    n_seeds_input = 400 
    target_features_input = 7
    verbose_input = input("Show detailed logs? (y/n, default: n): ").strip().lower() == 'y'

    start_time = time.time()

    most_stable_features_res, feature_freq_res, seed_details_res = multi_seed_stable_selection(
        X, y_series,
        n_seeds=n_seeds_input,
        target_features=target_features_input,
        start_seed=1,
        verbose=verbose_input
    )

    end_time = time.time()

    print(f"\nTotal execution time: {(end_time - start_time) / 60:.2f} minutes")
    print("\nFeature Selection Frequency (descending, showing top 17):")
    if feature_freq_res:
        for feature_item_print, count_print in feature_freq_res[:17]:
            percentage = (count_print / n_seeds_input) * 100
            print(f"{feature_item_print}: {count_print}/{n_seeds_input} ({percentage:.1f}%)")
    else:
        print("No features were selected or counted.")

    print(f"\nRecommended most stable {target_features_input}-feature set:")
    if most_stable_features_res:
        print(", ".join(most_stable_features_res))
    else:
        print("Could not determine a stable feature set.")

    print("\nProcessing complete! All results have been saved.")